In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install fasttext

In [0]:
import sys
import os
import csv
try:
    csv.field_size_limit(sys.maxsize)
except:
    import unicodecsv
    import ctypes
    unicodecsv.field_size_limit(int(ctypes.c_ulong(-1).value // 2))
    
import json
import datetime
import pandas as pd
import stat
import glob
import numpy as np
from scipy.sparse import lil_matrix
from datetime import datetime
from multiprocessing import Queue, Process
import collections
import multiprocessing
from copy import deepcopy
from tqdm import tqdm
import time

import pickle as pkl

from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import fasttext

import gensim
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.corpora.dictionary import Dictionary

In [0]:
# txt 
def get_txt_text(file, length):
    for index in range(length):
        text = file.readline()
    return text

def fill_dataframe(data_frame, classifications_df, classcode, abstract, claim, description):
    if classcode != "":
        # shrink the set to only_top_classes? TRUE/FALSE
        classcode = cut_down(classcode, 4, ['H', 'B', 'C'], False) # H, B, C
        data_frame.loc[data_frame.shape[0] + 1] = [abstract, claim, description, classcode]
        classifications_df = calculate_class_distribution(classcode, classifications_df)

def handle_patent_file(data_frame, classifications_df, path_filename):
    file = open(path_filename, "r")

    kind = get_txt_text(file, 1).strip()
    if kind == 'A1':
        classcode = get_txt_text(file, 1).strip()
        applicant = get_txt_text(file, 1).strip()

        abstract = get_txt_text(file, 1).strip()
        citations = get_txt_text(file, 1).strip()
        file.close()

        fill_dataframe(data_frame, classifications_df, classcode, abstract, None, None)
    elif kind == 'B1':
        classcode = get_txt_text(file, 1).strip()
        id_respective_document = get_txt_text(file, 1).strip()

        # abstract = get_txt_text(file, 1).strip()
        claim = get_txt_text(file, 1).strip()
        description = get_txt_text(file, 1).strip()
        # citations = get_txt_text(file, 1).strip()
        file.close()

        fill_dataframe(data_frame, classifications_df, classcode, None, claim, description)
    else:
        if kind == 'A1B1':
            print("eu_mix_patent !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        else:
            print("us_patent     !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        classcode = get_txt_text(file, 1).strip()
        applicant = get_txt_text(file, 1).strip()

        abstract = get_txt_text(file, 1).strip()
        claim = get_txt_text(file, 1).strip()
        description = get_txt_text(file, 1).strip()
        citations = get_txt_text(file, 1).strip()
        file.close()

        fill_dataframe(data_frame, classifications_df, classcode, abstract, claim, description)
    return get_patent_id(path_filename)

def handle_path_patent(data_frame, classifications_df, path):
    return list(map(lambda path_filename : handle_patent_file(data_frame, classifications_df, path_filename), get_list_files(path, 'txt')))

def load_data(source_path):
    print('###  reading patents  ###')
    """ load_data """
    data_frame = pd.DataFrame(columns=['abstract', 'claim', 'description', 'classification'])
    classifications_df = pd.DataFrame(columns=['class', 'count'])
    patent_ids = []

    patent_ids = list(map(lambda path : handle_path_patent(data_frame, classifications_df, path), source_path))
    patent_ids = get_flat_list(patent_ids)

    data_frame['id'] = data_frame.index
    classifications_df.sort_values(by=['count'], ascending=False, inplace=True, kind='quicksort')
    return patent_ids, data_frame, classifications_df

def handle_row(row, ids_list):
    if isinstance(row, pd.Series):
        try:
            id_, patent_id, text, classcodes = row.tolist()
        except:
            patent_id, text, classcodes = row.tolist()
        tokens = tokenize_text(text)
        if len(tokens) < 2:
            ids_list.append(patent_id)
        else:
            if isinstance(classcodes, str):
                temp_classcodes = tokenize_text(classcodes)
                for class_ in temp_classcodes:
                    if len(class_) == 4:
                        if class_[0] in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] and class_[1].isdigit() and class_[2].isdigit() and class_[3].isalpha():
                            pass
                        else:
                            ids_list.append(patent_id)
                            break
                    else:
                        ids_list.append(patent_id)
                        break
            else:
                print('not string')

def check_out_for_whitespaces(text):
    if isinstance(text, str):
        return ' '.join([element for element in tokenize_text(text) if len(element) > 2 and len(element) < 31])

def check_out_empty_texts_and_wrong_classcodes(data_frame):
    print('dataframe shape: ', data_frame.shape)
    ids_list = []
    data_frame.apply(lambda row : handle_row(row, ids_list), axis=1)  
    data_frame.set_index(data_frame['patent_id'], inplace=True)
    
    data_frame.drop(ids_list, axis=0, inplace=True)

    data_frame['text'] = data_frame['text'].apply(lambda text : check_out_for_whitespaces(text))
    print('ids_list: ', ids_list)
    print('dataframe shape: ', data_frame.shape)
    return data_frame

def clean_text(text):
    text = re.sub(r'\b\w{1,2}\b', '', text)
    text = ' '.join(text.split())
    return text

def get_final_df(patent_ids, temp_df, classif_type):
    classification_target = 'description_claim_abstract_title'

    data_frame = pd.DataFrame(columns=['id_', 'patent_id', 'text', 'classification'])

    if temp_df.shape[1] == 5:
        data_frame['patent_id'] = temp_df['patent_id']
    else:
        data_frame["patent_id"] = patent_ids

    # textual_information = temp_df["abstract"]
    # textual_information = temp_df["claim"]
    # textual_information = temp_df["description"]
    textual_information = temp_df["abstract"].str.cat(temp_df["claim"], sep =" ", na_rep=" ").str.cat(temp_df["description"], sep =" ", na_rep=" ")
    data_frame['text'] = textual_information

    # data_frame['text'] = data_frame['text'].apply(lambda text : clean_text(text))
    data_frame = check_out_empty_texts_and_wrong_classcodes(data_frame)

    classification_types = get_classifications(temp_df)
    data_frame["classification"] = classification_types[classif_type]
    return data_frame, classification_target, classif_type

In [0]:
# folder
def create_folder(destination_path):
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)

def get_list_files(path, extension):
    if extension:
        return glob.glob(path + '*.' + extension)
    return glob.glob(path)

def link_paths(root_path, ending_path):
    return os.path.join(root_path, ending_path)

def join_paths(root_path, ending_path):
    path = link_paths(root_path, ending_path)
    create_folder(path)
    return path

def get_root_location(ending_path):
    return join_paths('/content/drive/My Drive/Colab Notebooks/', ending_path)

In [0]:
# tool
def cut_down(classification, index, top_classes, only_top_classes):
    temp_list = tokenize_text(classification)
    if only_top_classes:
        new_classification_list = list(set(filter(lambda element : element[:index] in top_classes, temp_list)))
        new_classification = get_string_from_list(new_classification_list, ' ')
    else:
        new_classification_list = list(set(map(lambda element : element[:index], temp_list)))
        new_classification = get_string_from_list(new_classification_list, ' ')
    return new_classification

def calculate_class_distribution(classification, classifications_df):
    for _class in tokenize_text(classification):
        if classifications_df['class'].str.contains(_class).any():
            index = classifications_df.index[classifications_df['class'] == _class]
            classifications_df.loc[index[0], ['count']] += 1
        else:
            classifications_df.loc[classifications_df.shape[0] + 1] = [_class, 1]
    return classifications_df

def get_flat_list(list_of_lists):
    return [element for elements in list_of_lists for element in elements]

def tokenize_text(text):
    return text.split()

def tokenize_complex_text_in_set(text):
    return set(map(lambda word : word, tokenize_text(text)))

def get_set_from_index(index_set, data_set):
    set_ = np.ndarray(shape=(index_set.shape[0], data_set.shape[1]))
    for index, data_index in enumerate(index_set['text'].values):
        set_[index-1] = data_set[data_index-1]
    return set_, index_set['patent_id']

def get_string_from_list(list_, linking_string):
    return linking_string.join(element for element in list_ if isinstance(element, str))

def get_patent_id(string):
    index = string.rfind('/')
    return string[index+1:-4]

In [0]:
# word
def train_doc2vec(data_, model_path):
    cores = multiprocessing.cpu_count()
    model_dbow = Doc2Vec(dm=0, vector_size=150, negative=5, min_count=1, alpha=0.25, min_alpha=0.05, sample=0, workers=cores)
    model_dbow.build_vocab([x for x in tqdm(data_)])
    date = datetime.datetime.now().isoformat()
    model_dbow.save(link_paths(model_path, 'doc2vec_model '+date))
    return model_dbow

class Dov2VecHelper():
    def __init__(self):
        print('###  doc2vec_vectorizer  ###')
        self.labeled = []
        self.vectors = []

    def label_sentences(self, corpus, label_type):
        """
        Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
        We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
        a dummy index of the post.
        """
        self.labeled = []
        for i, v in enumerate(corpus):
            label = label_type + '_' + str(i)
            self.labeled.append(TaggedDocument(th.tokenize_text(v), [label]))
        return self.labeled

    def get_vectors(self, model, corpus_size, vectors_size, vectors_type):
        """
        Get vectors from trained doc2vec model
        :param doc2vec_model: Trained Doc2Vec model
        :param corpus_size: Size of the data
        :param vectors_size: Size of the embedding vectors
        :param vectors_type: Training or Testing vectors
        :return: list of vectors
        """
        self.vectors = np.zeros((corpus_size, vectors_size))
        for i in range(0, corpus_size):
            prefix = vectors_type + '_' + str(i)
            self.vectors[i] = model.docvecs[prefix]
        return self.vectors

def get_word2vec_model(data_, out_path):
    cores = multiprocessing.cpu_count()
    # results_word2vec_sg_1_size_150_hs_1_cbow_mean_1_negative_5_min_count_1_lpha_0.25_min_alpha_0
    # results_word2vec_sg_0_size_150_hs_1_cbow_mean_1_negative_12_min_count_1_lpha_0.25_min_alpha_0
    model_dbow = Word2Vec(data_, sg=1, size=150, hs=1, cbow_mean=1, negative=5, min_count=1, alpha=.25, min_alpha=.0, compute_loss=True, window=5, workers=cores)
    # model_dbow.build_vocab([x for x in tqdm(data_)])

    for epoch in range(30):
        model_dbow.train(utils_shuffle_rows([x for x in tqdm(data_)]), total_examples=len(data_), epochs=1)
        model_dbow.alpha -= 0.002
        model_dbow.min_alpha = model_dbow.alpha

    date = datetime.datetime.now().isoformat()
    model_dbow.save(link_paths(out_path, 'word2vec_model ' + date))
    return model_dbow

class Word2VecHelper():
    def __init__(self):
        print('###  word2vec_vectorizer  ###')
        self.tokens = []
        self.mean = []

        self.vectors = []

    def w2v_tokenize_text(self, text):
        self.tokens = []
        for word in tokenize_text(text):
            if len(word) < 2:
                continue
            self.tokens.append(word)
        return self.tokens

    def word_averaging(self, wv, words):
        all_words = set()
        self.mean = []

        for word in words:
            if isinstance(word, np.ndarray):
                self.mean.append(word)
            elif word in wv.vocab:
                self.mean.append(wv.syn0norm[wv.vocab[word].index])
                all_words.add(wv.vocab[word].index)

        if not self.mean:
            print("POSSIBLE ERROR IN WORD2VECHELPER: cannot compute similarity with no input %s", words)
            return np.zeros(wv.vector_size,)

        self.mean = gensim.matutils.unitvec(np.array(self.mean).mean(axis=0)).astype(np.float32)
        return self.mean

    def  word_averaging_list(self, wv, text_list):
        return np.vstack([self.word_averaging(wv, post) for post in text_list])

    def reduce_dimensions(model, X_data):
        # num_dimensions = 2  # final num dimensions (2D, 3D, etc)

        vectors = [] # positions in vector space
        new_element = []
        # labels = [] # keep track of words to label our data again later
        for element in X_data:
            print("element: ", element)
            for word in element:
                new_element.append(model.wv[word])
                # labels.append(word)
            vectors.append(new_element)

        # convert both lists into numpy vectors for reduction
        vectors = np.asarray(vectors)
        # labels = np.asarray(labels)

        # reduce using t-SNE
        # vectors = np.asarray(vectors)
        # tsne = TSNE(n_components=num_dimensions, random_state=0)
        # vectors = tsne.fit_transform(vectors)

        x_vals = [v[0] for v in vectors]
        # y_vals = [v[1] for v in vectors]
        return x_vals #, y_vals, labels

In [0]:
# classification
def get_distinct_class_substrings(classification_list, first_int, second_int):
    list_ = []
    for element in classification_list:
        for class_ in tokenize_text(element):
            if class_[first_int:second_int] not in list_:
                list_.append(class_[first_int:second_int])
    list_.sort()
    return list_

# def apply_method_to_get_classes(element):
#     list_ = list(map(lambda class_ : class_, th.tokenize_text(element)))
#     return " ".join(item for item in list(set(list_)))

# def get_alternative_class_substrings(classification_list, first_int, second_int):
#     list_ = list(map(lambda element : apply_method_to_get_classes(element), classification_list))
#     return list_

def get_class_substrings(classification_list, first_int, second_int):
    list_ = []
    for element in classification_list:
        string = ""
        for class_ in tokenize_text(element):
            if class_[first_int:second_int] not in string:
                string += class_[first_int:second_int] + ' '
        list_.append(string)
    return list_

def get_classifications(temp_df):
    ## Load utility data
    classifications = temp_df['classification'].tolist()

    valid_sections = get_distinct_class_substrings(classifications, 0, 1)
    valid_classes = get_distinct_class_substrings(classifications, 1, 3)
    valid_subclasses = get_distinct_class_substrings(classifications, 3, 4)

    sections = get_class_substrings(classifications, 0, 1)
    classes = get_class_substrings(classifications, 0, 3)
    subclasses = get_class_substrings(classifications, 0, 4)

    classification_types = {
        "sections": sections,
        "classes": classes,
        "subclasses": subclasses
    }
    return classification_types

def get_train_test_from_data(X, Y):
    return train_test_split(X, Y, random_state=0, test_size=.2, shuffle=True)

def get_train_test_from_dataframe(data_frame):
    return train_test_split(data_frame, random_state=0, test_size=.2, shuffle=True)

def save_sets(sets_location, train_data, test_data, val_data, train_labels, test_labels, val_labels, settings, date='01-01-2020'):
    try:  
        sets_location = join_paths(sets_location, 'tensorflow ' + tf.version.VERSION)
        actual_sets_location = join_paths(sets_location, date)
        with open(link_paths(actual_sets_location, 'training_set '+date+'.pkl'), "wb") as f:
            pkl.dump([train_data, train_labels], f)
        with open(link_paths(actual_sets_location, 'testing_set '+date+'.pkl'), "wb") as f:
            pkl.dump([test_data, test_labels], f)
        with open(link_paths(actual_sets_location, 'validation_set '+date+'.pkl'), "wb") as f:
            pkl.dump([val_data, val_labels], f)
        with open(link_paths(actual_sets_location, 'settings '+date+'.pkl'), "wb") as f:
            pkl.dump(settings, f)
    except:
        print('A problem occurred while saving the sets!')

def load_sets(sets_location, date='01-01-2020'):
    try:
        sets_location = join_paths(sets_location, 'tensorflow ' + tf.version.VERSION)
        actual_sets_location = join_paths(sets_location, date)
        with open(link_paths(actual_sets_location, 'training_set '+date+'.pkl'), "rb") as f:
            train_data, train_labels = pkl.load(f)
        with open(link_paths(actual_sets_location, 'testing_set '+date+'.pkl'), "rb") as f:
            test_data, test_labels = pkl.load(f)
        with open(link_paths(actual_sets_location, 'validation_set '+date+'.pkl'), "rb") as f:
            val_data, val_labels = pkl.load(f)
        with open(link_paths(actual_sets_location, 'settings '+date+'.pkl'), "rb") as f:
            try:
                settings = pkl.load(f)
                # classes, n_classes, vocab_processor, len_vocabulary = pkl.load(f)
            except:
                settings = [None, None, None, None]
        return train_data, test_data, val_data, train_labels, test_labels, val_labels, settings
    except:
        print('A problem occurred while loading the sets!')
        return None, None, None, None, None, None, None

def save_data_frame(script_key, data_frame, csvfile):
    path_to_csv = get_root_location('model_dataframe/')
    date = datetime.now().isoformat()
    if csvfile:
        output_path = fh.link_paths(path_to_csv, csvfile)
    else:
        output_path = fh.link_paths(path_to_csv, 'dataframe '+date+'.csv')
    print('output_path: ', output_path)
    
    data_frame.to_csv(output_path, index=False, sep=',', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

def load_data_frame(script_key, csvfile):
    path_to_csv = get_root_location('model_dataframe/')
    if csvfile:
        input_path = link_paths(path_to_csv, csvfile)
    else:
        input_path = link_paths(path_to_csv, 'dataframe '+date+'.csv')
    print('input_path: ', input_path)
    
    data_frame = pd.read_csv(input_path, sep=',', quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ", header=None, engine='python')
    data_frame.columns = ['patent_id', 'text', 'classification']
    
    # data_frame['text'] = data_frame['text'].apply(lambda text : clean_text(text))
    data_frame = check_out_empty_texts_and_wrong_classcodes(data_frame)
    
    classification_df = pd.DataFrame(columns=['class', 'count'])
    data_frame['classification'].apply(lambda classcode : calculate_class_distribution(classcode, classification_df))
    return data_frame, classification_df

# as a list of strings, which are consequently made of labels (as above separated by an empty space)
def apply_simple_binarizer(classification, classes):
    print('###  label_binarizer  ###')
    binarized_classification = label_binarize(classification, classes=classes)
    return binarized_classification, classes, binarized_classification.shape[1]

# TODO: may be the same as the simple
def apply_label_binarizer(classification):
    lb = LabelBinarizer()
    return lb.fit_transform(classification)

def apply_multilabel_binarizer(data_frame):
    print('###  multi_label_binarizer  ###')
    ################################################ classification: from text to sparse binary matrix [[0, 1, 0],[1, 0, 1]]
    temp_classification = data_frame.apply(lambda row : tokenize_complex_text_in_set(row['classification']), axis=1)
    df_to_list = temp_classification.tolist()
    mlb = MultiLabelBinarizer()
    mlb.fit(df_to_list)
    classes = list(mlb.classes_)
    return mlb.transform(df_to_list), classes, len(classes)

def apply_tfidf_vectorizer_fit(data_frame):
    print('###  tfidf_vectorizer_text  ###')
    vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
    vectorizer.fit(data_frame['text'].apply(lambda x : np.str_(x)))
    return vectorizer

def apply_tfidf_vectorizer_transform(text, vectorizer):
    return vectorizer.transform(text.apply(lambda x : np.str_(x)))

def apply_word2vec_word_averaging(data_frame):
    print('###  word2vec_word_averaging_vectorizer  ###')
    helper_word2vec = Word2VecHelper()
    text_tokenized = data_frame.apply(lambda row : helper_word2vec.w2v_tokenize_text(row['text']), axis=1).values
    if False:
        try:
            path_ = join_paths(current_path[:current_path.rfind('/', 0, -8)], "/data/GoogleNews-vectors-negative300.bin")
            print("google vectors : ", path_)
            wv = gensim.models.KeyedVectors.load_word2vec_format(path_, binary=True)
        except:
            print("unable to find word2vec model from google, downloading...")
            wv = gensim.models.KeyedVectors.load_word2vec_format("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", binary=True)
            print("done...")
    else:
        model_path = join_paths(current_path[:current_path.rfind('/', 0, -8)], "data/vectorizer_models")
        model = get_word2vec_model(text_tokenized, model_path)
        wv = model.wv
    wv.init_sims(replace=True)
    return helper_word2vec.word_averaging_list(wv, text_tokenized)

def apply_doc2vec(data_frame, type_, doc2vec_path):
    Y, classes, n_classes = apply_classification_vectorizer(type_, data_frame)
    print('finisched class vect')
    X_train, X_test, y_train, y_test = get_train_test_from_data(data_frame, Y)
    print('finisched get train test')
    patent_ids = X_train['patent_id']
    print('finisched ids')
    helper_doc2vec = Dov2VecHelper()
    print('finisched doc2vec helpers')
    X_train = helper_doc2vec.label_sentences(X_train['text'], 'Train')
    print('finisched label sent')
    X_test = helper_doc2vec.label_sentences(X_test['text'], 'Test')
    print('finisched label sent')
    all_data = X_train + X_test
    print('finisched all data')
    model_dbow = train_doc2vec(all_data, X_train, X_test, y_train, y_test, n_classes, doc2vec_path)
    print('finisched train doc2vec')
    train_vectors_dbow = helper_doc2vec.get_vectors(model_dbow, len(X_train), 150, 'Train')
    print('finisched get vect')
    test_vectors_dbow = helper_doc2vec.get_vectors(model_dbow, len(X_test), 150, 'Test')
    print('finisched get vect')
    return train_vectors_dbow, test_vectors_dbow,  y_train, y_test, classes, n_classes, patent_ids

def apply_standard_vectorizer(data_frame, type_):
    Y, classes, n_classes = apply_classification_vectorizer(type_, data_frame)
    X, vocab_processor, len_vocabulary = apply_vocabulary_processor(data_frame['text'])
    # a list of the words used in the text, identified by a unique number for each different word

    X_train, X_test, y_train, y_test = get_train_test_from_data(X, Y)

    print("Vocabulary Size: {:d}".format(len_vocabulary))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_test)))
    return X_train, X_test, y_train, y_test, classes, n_classes, vocab_processor, len_vocabulary

def set_string_for_fasttext(item):
    return str(item).strip().replace(' or ', ' ').replace(', or ', ' ').replace(', ', ' ').replace(',', ' __label__').replace('$$', ' ').replace('$', ' ').replace(' ', ' __label__').replace('___', '__')

def apply_classification_vectorizer(type_, data_frame):
    classification = data_frame['classification']
    if type_ == 'simple':
        classes = ['H', 'B', 'C'] # useful if i need to shrink the set of classes
        return apply_simple_binarizer(classification, classes)
    elif type_ == 'label_binarizer':
        return apply_label_binarizer(classification), None, 0
    elif type_ == 'multi_label':
        return apply_multilabel_binarizer(data_frame)
    elif type_ == 'fasttext':
        return ['__label__'+set_string_for_fasttext(item) for item in data_frame['classification']], None, 0
    return classification, None, 0

def apply_df_vectorizer(data_frame, type_, classification_type, model_name):
    model_path = '/Users/elio/Desktop/Patent-Classification/data/vectorizer_models/'
    # type_ = "doc2vec"
    # classification_type = "multi_label"
    if type_ == 'doc2vec':
        X_train, X_test, y_train, y_test, classes, n_classes, patent_ids = apply_doc2vec(data_frame, classification_type, model_path)
        # array - not numpy
        print('finisched apply doc2vec')
        vocab_processor, len_vocabulary = None, 0
    elif type_ == 'standard':
        standard_results = apply_standard_vectorizer(data_frame, classification_type)
        X_train, X_test, y_train, y_test, classes, n_classes, vocab_processor, len_vocabulary = standard_results
        patent_ids = data_frame['patent_id']
    else:
        Y_vect, classes, n_classes = apply_classification_vectorizer(classification_type, data_frame)
        vocab_processor, len_vocabulary = None, 0
        if type_ == 'tfidf_fit_transform': # not sure it remains the same order
            X_vect = pd.DataFrame({'text' : data_frame['text'], 'patent_id' : data_frame['patent_id']})
            X_train, X_test, y_train, y_test = get_train_test_from_data(X_vect, Y_vect)

            patent_ids = X_train['patent_id']
            print('almost started tfidf')
            vectorizer = apply_tfidf_vectorizer_fit(data_frame)
            print('applied tfidf')
            X_train = apply_tfidf_vectorizer_transform(X_train['text'], vectorizer)
            print('transformed train')
            X_test = apply_tfidf_vectorizer_transform(X_test['text'], vectorizer)
            print('transformed test')
            # csr matrix
        elif type_ == 'word2vec': # required google pre-trained vectors, it starts downloading if you don't have
            X_vect = apply_word2vec_word_averaging(data_frame)
            # ndarray
            print('finished word2vec word averaging')
            text_indexes = data_frame.index.values

            X_vect_temp = pd.DataFrame({'text' : text_indexes, 'patent_id' : data_frame['patent_id']})
            X_train_temp, X_test_temp, y_train, y_test = get_train_test_from_data(X_vect_temp, Y_vect)

            X_train, patent_ids = get_set_from_index(X_train_temp, X_vect)
            X_test, _ = get_set_from_index(X_test_temp, X_vect)
            print('finished word2vec vectorizer')
        else:
            data_frame.drop(columns=['classification'])
            X_train, X_test, y_train, y_test = get_train_test_from_data(data_frame, Y_vect)
            return X_train, X_test, y_train, y_test, classes, n_classes, vocab_processor, len_vocabulary

    return X_train, X_test, y_train, y_test, classes, n_classes, vocab_processor, len_vocabulary

def get_fasttext_classifier_information(text):
    return text.split('.')[1]

def get_metrics_values(metrics):
    return list(map(lambda tuple_ : apply_method_to_create_metrics(tuple_), metrics))

def get_sequential_layers_values(parameters):
    return [list(map(lambda token : token, th.tokenize_text(parameters[:-1])))]

def get_sequential_LSTM_metrics_values(metrics):
    micro = ["precision:"+str(metrics["precision_micro"]), "recall:"+str(metrics["recall_micro"]), "f1:"+str(metrics["f1_micro"])]
    macro = ["precision:"+str(metrics["precision_macro"]), "recall:"+str(metrics["recall_macro"]), "f1:"+str(metrics["f1_macro"])]
    tops = ["top_1:"+str(metrics["top_1"]), "top_3:"+str(metrics["top_3"]), "top_5:"+str(metrics["top_5"])]
    return [str(metrics["coverage_error"]), micro, macro, tops]

def get_parameters_values(parameters):
    parameters.replace(" ", "")
    return list(map(lambda token : token.split('=')[1], parameters.split(',')))

def get_parameters_list(model, metrics, parameters, classif_approach, classif_level, classif_type, dataset_location):
    list_ = [model]
    list_ += list(map(lambda item : item, metrics))
    list_ += [classif_approach, classif_level, classif_type]
    list_ += list(map(lambda item : item, parameters))
    list_ += [dataset_location]
    return list_

def get_saving_dataframe(model_key, metrics_values, parameters_values, classif_approach, classif_level, classif_type, dataset_location):
    columns_list = get_parameters_list('model_name', settings[model_key]["metrics"], settings[model_key]["parameters"], 'approach', 'level', 'type', 'dataset_location')
    values_list = get_parameters_list(model_key, metrics_values, parameters_values, classif_approach, classif_level, classif_type, dataset_location)

    data_frame = pd.DataFrame(columns=columns_list)
    data_frame.loc[data_frame.shape[0] + 1] = values_list
    return data_frame

def write_dataframe_as_csv(data_frame, path_to_csv):
    if os.path.isfile(path_to_csv):
        with open(path_to_csv, 'a') as f:
            data_frame.to_csv(f, sep=',', header=False)
    else:
        data_frame.to_csv(path_to_csv, sep=',', header=True)

settings = {
            "FastText" : {
                          "results_file_name" : "data/models_results/sequantial_results.csv",
                          "metrics" : ['precision', 'recall'],
                          "parameters" : ['parameters'],
            }
}

def save_results(model_key, metrics, parameters, classif_approach, classif_level, classif_type, dataset_location):
    print('###  saving_results  ###')
    root_location = get_root_location('data/fasttext_outcome/')
    path_to_csv = link_paths(join_paths(root_location, 'model_results'), 'results.csv')

    if model_key == 'Sequential':
        metrics = list(map(lambda metric : [metric], metrics))
        # metrics = [[metric] for metric in metrics]
        metrics_values = get_metrics_values(metrics)
        parameters_values = get_sequential_layers_values(parameters)
    elif model_key == 'Sequential Test':
        metrics = [[-1], [-1], [-1], # metrics['loss'], metrics['accuracy'], metrics['mse'],
                   [metrics['precision_micro'], metrics['recall_micro'], metrics['f1_micro']],
                   [metrics['precision_macro'], metrics['recall_macro'], metrics['f1_macro']]]
        metrics_values = get_metrics_values(metrics)
        parameters_values = get_sequential_layers_values(parameters)
    elif model_key == 'FastText':
        metrics_values = metrics[1:3]
        parameters_values = [parameters]
    elif model_key == 'Sequential_LSTM':
        temp_metrics = metrics
        del temp_metrics["total_positive"], temp_metrics["average_num_of_labels"]
        metrics_values = get_sequential_LSTM_metrics_values(temp_metrics)
        parameters_values = get_sequential_layers_values(parameters)
    else:
        metrics_values = get_metrics_values(metrics)
        parameters_values = get_parameters_values(parameters)

    data_frame = get_saving_dataframe(model_key, metrics_values, parameters_values, classif_approach, classif_level, classif_type, dataset_location)
    write_dataframe_as_csv(data_frame, path_to_csv)

In [0]:
# metrics helper
def display_directly_metrics(algorithm, accuracy, precision, recall, f1):
    print('@@@  ', algorithm ,'  @@@')

    print('test accuracy : {0:0.5f}'.format(accuracy))
    print('test precision : {0:0.5f}'.format(precision))
    print('test recall : {0:0.5f}'.format(recall))
    print('test f1 : {0:0.5f}'.format(f1))

In [0]:
script_key = "fasttext classify"

def preprocessing_data_for_fasttext(data_frame, text_vectorizer, class_vectorizer):
    root_location = get_root_location('data/fasttext_outcome/')

    data_frame['text'] = data_frame['text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)

    model_name = text_vectorizer+'/'+class_vectorizer+'/FastText'
    X_train, X_test, Y_train, Y_test, _, _, _, _ = apply_df_vectorizer(data_frame, text_vectorizer, class_vectorizer, model_name)

    X_train, X_val, Y_train, Y_val = get_train_test_from_data(X_train, Y_train)
    # try:
    if not isinstance(X_train, pd.DataFrame):
        train = pd.DataFrame(data=X_train)
        test = pd.DataFrame(data=X_test)
        val = pd.DataFrame(data=X_val)
    else:
        train = X_train
        test = X_test
        val = X_val

    train.loc[:, 1] = Y_train
    test.loc[:, 1] = Y_test
    val.loc[:, 1] = Y_val

    print(train.head(5))
    
    train.drop(columns=['patent_id'], inplace=True)
    test.drop(columns=['patent_id'], inplace=True)
    val.drop(columns=['patent_id'], inplace=True)
    
    val.to_csv(link_paths(root_location, 'validating set.csv'), index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="",escapechar=" ")
    train.to_csv(link_paths(root_location, 'training set.csv'), index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="",escapechar=" ")
    test.to_csv(link_paths(root_location, 'testing set.csv'), index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

    print('csv saved')
    # except:
    #     print('a problem occurred while trying to store the dataframes')

    #     val = pd.DataFrame({'text': X_val, 'classification': Y_val})
    #     train = pd.DataFrame({'text': X_train, 'classification': Y_train})
    #     test = pd.DataFrame({'text': X_test, 'classification': Y_test})

    #     data_frame.to_csv(link_paths(root_location, 'dataframe.csv'), index=False, sep=' ', header=False,quoting=csv.QUOTE_NONE,quotechar="",escapechar=" ")

    #     val.to_csv(link_paths(root_location, 'validating set.csv'), index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="",escapechar=" ")
    #     train.to_csv(link_paths(root_location, 'training set.csv'), index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="",escapechar=" ")
    #     test.to_csv(link_paths(root_location, 'testing set.csv'), index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
        
    #     print('csv saved')

In [0]:
def apply_fasttext(data_frame, text_vectorizer, class_vectorizer, classif_level, classif_type, dataset_location):
    root_location = get_root_location('data/fasttext_outcome/')

    fasttext_location = join_paths(root_location, 'fasttext_models/')
    word2vec_location = join_paths(root_location, 'word2vec_models/')
    timestamp = datetime.now().isoformat()

    # model = fasttext.train_supervised(input=link_paths(root_location, 'training set.csv'), autotuneValidationFile=link_paths(root_location, 'validating set.csv'), verbose=3)

    model = fasttext.train_supervised(input=link_paths(root_location, 'training set.csv'), dim=300, minCount=1, minn=0, maxn=0, epoch=30, lr=.618225, loss='softmax', bucket=937478, wordNgrams=3, 
                                      pretrainedVectors='/content/drive/My Drive/Colab Notebooks/data/crawl-300d-2M.vec')
    parameters = '[dim=300, minCount=1, minn=0, maxn=0, epoch=30, lr=.618225, loss=softmax, bucket=937478, wordNgrams=3]'

    correct = 0
    total = 0

    y_pred = np.array([])
    texts = pd.read_csv(link_paths(root_location, 'training set.csv'))
    for index, row in texts.iterrows():
        if isinstance(row, pd.Series):
            text = row.tolist()
            # print('shrink ', text[-15:])
            possible_labels = text[-15:][0]
            y_predicted, metrics_array = model.predict(text, k=-1, threshold=.05)
            np.append(y_pred, y_predicted)
            total += 1
            # print(tokenize_text(possible_labels))
            lbl = list(filter(lambda x: x.lower().startswith("__label__"), tokenize_text(possible_labels)))
            # print(lbl, y_predicted[0])
            total += len(lbl)-1
            # print('toal: ', total)
            correct += len(list(set(y_predicted[0]).intersection(lbl)))

    results = model.test(link_paths(root_location, 'testing set.csv'), k=-1)

    result_top_5 = model.test(link_paths(root_location, 'testing set.csv'), k=5)
    result_top_3 = model.test(link_paths(root_location, 'testing set.csv'), k=3)
    result_top_1 = model.test(link_paths(root_location, 'testing set.csv'), k=1)

    classifier_name = get_fasttext_classifier_information(str(model))
    model_name = text_vectorizer+'/'+class_vectorizer+'/'+classifier_name

    display_directly_metrics('k=-1 '+classifier_name, -1, results[1], results[2], 2*(results[1]*results[2])/(results[1]+results[2]))
    display_directly_metrics('k= 5 '+classifier_name, -1, result_top_5[1], result_top_5[2], 2*(result_top_5[1]*result_top_5[2])/(result_top_5[1]+result_top_5[2]))
    display_directly_metrics('k= 3 '+classifier_name, -1, result_top_3[1], result_top_3[2], 2*(result_top_3[1]*result_top_3[2])/(result_top_3[1]+result_top_3[2]))
    display_directly_metrics('k= 1 '+classifier_name, -1, result_top_1[1], result_top_1[2], 2*(result_top_1[1]*result_top_1[2])/(result_top_1[1]+result_top_1[2]))

    save_results(classifier_name, results, parameters, model_name, classif_level, classif_type, dataset_location)

    print(y_pred)

    print(correct/total, correct, total)

    print(model.labels)

In [0]:
def shrink_vocabulary(row, vocabulary, data_frame, ids_list):
    if isinstance(row, pd.Series):
        patent_id, text, classification = row.tolist()

        new_tokens = [element for element in text if element in vocabulary]
        if new_tokens != [] or len(new_tokens) > 2:
            data_frame.loc[data_frame.shape[0]+1] = [patent_id, ' '.join(element for element in new_tokens), classification]
        else:
            ids_list.append(patent_id)

def further_preprocessing_phase(temp_data_frame):
    temp_data_frame['text'] = temp_data_frame['text'].apply(lambda text: th.tokenize_text(text) if text != None else '')
    # textlist = temp_data_frame['text'].to_numpy()
    textlist = temp_data_frame['text'].tolist()

    # if it raises an exeption could be the empty texts
    patent_dictionary = Dictionary(textlist)
    corpus = [patent_dictionary.doc2bow(text) for text in textlist]

    print('original dictionary size: ', len(patent_dictionary))

    vocab_tf={}
    for i in corpus:
        for item, count in dict(i).items():
            if item in vocab_tf:
                vocab_tf[item]+=int(count)
            else:
                vocab_tf[item] =int(count)

    remove_ids=[]
    no_of_ids_below_limit=0
    for id,count in vocab_tf.items():
        if count<=5:
            remove_ids.append(id)
    patent_dictionary.filter_tokens(bad_ids=remove_ids)

    patent_dictionary.filter_extremes(no_below=0)
    patent_dictionary.filter_n_most_frequent(30)

    print('parsed dictionary size: ', len(patent_dictionary))

    vocabulary = list(patent_dictionary.token2id.keys())

    ids_list = []
    data_frame = pd.DataFrame(columns=['patent_id', 'text', 'classification'])
    temp_data_frame.apply(lambda row : shrink_vocabulary(row, vocabulary, data_frame, ids_list), axis=1)
    print(len(ids_list))
    data_frame.set_index(data_frame['patent_id'], inplace=True)
    data_frame.drop(ids_list, axis=0, inplace=True)
    return data_frame

def shrink_to_sectors(classcode_list):
    new_list = []
    for class_ in classcode_list:
        if class_[0] not in new_list:
            new_list.append(class_[0])
    return ' '.join(element for element in new_list)

if __name__ == '__main__':

    # source_path = ['/Users/elio/Desktop/Patent-Classification/data/test_classification/cleaned/B/']
    source_path = ['/content/drive/My Drive/Colab Notebooks/data/test_classification/B 1500 patents/']
    source_path = ['/content/drive/My Drive/Colab Notebooks/data/test_classification/B 500 patents/']

    start = time.time()

    text_vectorizer = 'None'
    class_vectorizer = 'fasttext'
    classif_level = 'description_claim_abstract_title'
    classif_type = 'subclasses'

    # patent_ids, temp_df, classifications_df = load_data(source_path)
    # data_frame, classif_level, classif_type = get_final_df(patent_ids, temp_df, classif_type)

    csvfile = 'training_2000_2004_cleaned.csv'
    # csvfile = 'training_2000_2004_cleaned_test.csv'
    csvfile = 'training_2000_2004_cleaned_test_smaller.csv'

    classif_level, classif_type = 'description_claim_abstract_title', 'subclasses'
    training_df, classification_df = load_data_frame(script_key, csvfile)
    
    csvfile = 'testing_2005_cleaned.csv'
    # csvfile = 'testing_2005_cleaned_test.csv'
    csvfile = 'testing_2005_cleaned_test_smaller.csv'
    testing_df, classification_df = load_data_frame(script_key, csvfile)

    data_frame = pd.concat([training_df, testing_df])
    patent_ids = data_frame['patent_id'].tolist()
    
    print('dataframe shape: ', data_frame.shape)

    data_frame = further_preprocessing_phase(data_frame)
    # data_frame['classification'] = data_frame['classification'].apply(lambda classcode : shrink_to_sectors(tokenize_text(classcode)))

    preprocessing_data_for_fasttext(data_frame, text_vectorizer, class_vectorizer)

    apply_fasttext(_, text_vectorizer, class_vectorizer, classif_level, classif_type, source_path)

    print("end fasttext classification step, execution time: ", time.time()-start)

# TODO list:
# try all the state-of-the art and maybe we can combine two of them.
#
# little trouble: i have to save two CSVs (one for testing one for training) in order to feed the model with them. It doesn't work directly with dataframes. is it normal?
#
# quite big trouble: the only way to specify the model for learning the word representation (cbow or skipgram) is using the unsupervised learning with the argument "model". There isn't the possibility to add it to the supervised. if i apply the vectorization before it works with impressively much less time.
#
# k means for? predictions
#
# gensim ha a fasttext implementation